In [ ]:
!pip install transformers
import pandas as pd
import os
import csv
import numpy as np 
import torch
from transformers import BertTokenizer, BertForMaskedLM
from google.colab import files
data = files.upload() 


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()

In [ ]:
def import_function(filename):
  df = pd.read_csv(filename)
  oldName = df.columns[0]
  df = df.rename(columns={oldName:'MaskedText'})
  print(df.head())
  df = df.head(5)
  return df

input_df = import_function(filename = '/content/MaskedDataEMPTYMASK .csv' )

In [ ]:
# Guess what's behind the mask
def unmask_sentence(index, text, tokenizer, model, top_k=5):

  text = "[CLS] %s [SEP]"%text
  tokenized_text = tokenizer.tokenize(text)
  masked_index = tokenized_text.index("[MASK]")
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  tokens_tensor = torch.tensor([indexed_tokens])

  column_names = ['Index','Sentence','1_word','1_weight','2_word','2_weight','3_word','3_weight','4_word','4_weight','5_word','5_weight']  
  df  = pd.DataFrame(columns = column_names)
  df['Sentence']= [text]
  df['Index']= [index]
  print(text)

  with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]  
  probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
  top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)
    
  for i, pred_idx in enumerate(top_k_indices):
    predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
    best_guess=predicted_token
    token_weight = float(top_k_weights[i])
    if i==0:
      df['1_word']= [predicted_token]
      df['1_weight'] =[token_weight]
    if i==1:
      df['2_word']= [predicted_token]
      df['2_weight'] =[token_weight]
    if i==2:
      df['3_word']= [predicted_token]
      df['3_weight'] =[token_weight]
    if i==3:
      df['4_word']= [predicted_token]
      df['4_weight'] =[token_weight]
    if i==4:
      df['5_word']= [predicted_token]
      df['5_weight'] =[token_weight]
    
  return df
  


In [ ]:
for i, text  in enumerate(input_df['MaskedText']):
  if i<5:
   print(i)
   print(text)

In [ ]:
def get_output_from_text(Index,enter_input_text):
  if enter_input_text.split(" ")[-1]=="[MASK]":
    enter_input_text+=" [MASK]" # preventing punctuation predictions for final mask
  column_names = ['Index','Sentence','1_word','1_weight','2_word','2_weight','3_word','3_weight','4_word','4_weight','5_word','5_weight']  
  sent_and_top5  = pd.DataFrame(columns = column_names)
  
  while "[MASK]" in enter_input_text:
    sent_and_top5_add=unmask_sentence(Index,enter_input_text, tokenizer, model)
    sent_and_top5 = pd.concat([sent_and_top5,sent_and_top5_add])
    words=enter_input_text.split(" ")
    for i in range(len(words)):
      if words[i]=="[MASK]":
        words[i]="unmasked"
        break
    enter_input_text=' '.join(words)

  
  return sent_and_top5

# input_df is now the input
# input_df is MaskedDatacsvfile
for i, text in enumerate(input_df['MaskedText']):
    if i == 0:
      output_df = get_output_from_text(i,text)
    else:
      addition = get_output_from_text(i,text)
      output_df = pd.concat([output_df,addition])







In [ ]:
def export_function(df, filename):
  df.to_csv(filename)

export_function(output_df, filename ='/content/UnmaskedJobPostings.csv')

In [ ]:
output_df